In [ ]:
import requests
import json
import configparser

from os import path
from PIL import Image
import IPython.display as display

In [ ]:
g_debugLog = False

class ALALogger:
    def log(in_log_self, in_str):
        print(f"{in_log_self}: log: {in_str}")
        
    def debugLog(in_log_self, in_str):
        if g_debugLog:
            ALALogger.log(in_log_self, f": debug: {in_str}")
        
class ALAResourceManager:
    def __init__(self, in_ALAMain):
        self._ALAMain = in_ALAMain
    
    def getProfileIconImage(self, in_profile_icon_id):
        return Image.open(path.join(self._ALAMain._root_dir, self._ALAMain._data_dragon_dir_rel, f"10.16.1/img/profileicon/{in_profile_icon_id}.png"))
    
    def getRankEmblem(self, in_tier):
        return Image.open(path.join(self._ALAMain._root_dir, f"Assets/ranked-emblems/Emblem_{(in_tier).capitalize()}.png"))
    
class RiotAPICaller:
    def __init__(self, in_ALAMain):
        self._ALAMain = in_ALAMain
        self._API_KEY = self._ALAMain._API_KEY
        
    def runGetSubdomain(self, in_subdomain_with_params, in_region, in_additional_params_str = ''):
        get_url = f"https://{in_region}.api.riotgames.com/lol/{in_subdomain_with_params}?api_key={self._API_KEY}{in_additional_params_str}"
        get_response = requests.get(get_url)
        ALALogger.debugLog(self, f"request:{get_url} returned with status code: {get_response.status_code}")
        if get_response.status_code == 200:
            return get_response.json()
        else:
            ALALogger.log(self, f"request:{get_url} returned with status code: {get_response.status_code}")
            return None
    
    # RiotAPI: Champion Mastery
    def runGetChampionMasteriesBySummonerId(self, in_encrypted_summoner_id, in_region):
        subdomain_with_params = f"champion-mastery/v4/champion-masteries/by-summoner/{in_encrypted_summoner_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    # RiotAPI: League
    def runGetSummonerLeagueBySummonerId(self, in_encrypted_summoner_id, in_region):
        subdomain_with_params = f"league/v4/entries/by-summoner/{in_encrypted_summoner_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    def runGetLeagueEntries(self, in_queue, in_tier, in_division, in_region, in_page):
        subdomain_with_params = f"league/v4/entries/{in_queue}/{in_tier}/{in_division}"
        return self.runGetSubdomain(subdomain_with_params, in_region, f"&page={in_page}")
    
    # RiotAPI: Match
    def runGetMatchListByAccount(self, in_encrypted_account_id, in_region):
        subdomain_with_params = f"match/v4/matchlists/by-account/{in_encrypted_account_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)

    def runGetMatchInfoByMatchId(self, in_match_id, in_region):
        subdomain_with_params = f"match/v4/matches/{in_match_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)

    def runGetMatchTimelineDataByMatchID(self, in_match_id, in_region):
        subdomain_with_params = f"match/v4/timelines/by-match/{in_match_id}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
    # RiotAPI: Summoner
    def runGetSummonerByName(self, in_summoner_name, in_region):
        subdomain_with_params = f"summoner/v4/summoners/by-name/{in_summoner_name}"
        return self.runGetSubdomain(subdomain_with_params, in_region)
    
class ALAMain:
    def __init__(self):
        # Read config file to get data locations.
        config = configparser.ConfigParser()
        config.read('config.ini')
        self._root_dir = config['DEFAULT']['root_dir']
        riot_api_key_file_rel = config['DEFAULT']['riot_api_key_file_rel']
        self._data_dragon_dir_rel = config['DEFAULT']['data_dragon_dir_rel']
        
        # Load RiotAPI key from file.
        with open(path.join(self._root_dir, riot_api_key_file_rel)) as f:
            self._API_KEY = json.loads(f.readlines()[0])["API_KEY"]
        
        self._riot_api_caller = RiotAPICaller(self)
        self._resource_man = ALAResourceManager(self)
        
        # load champion key map.
        self._champions = []
        self._id_to_champ = {}
        with open(path.join(self._root_dir, self._data_dragon_dir_rel, f"10.16.1/data/en_US/champion.json"), encoding='utf-8') as f:
            self._champions = json.loads(f.readlines()[0])['data']
        for ch in self._champions:
            self._id_to_champ[int(self._champions[ch]['key'])] = ch
            
    def getSummonerInfo(self, in_summoner_name, in_region):
        summoner = self._riot_api_caller.runGetSummonerByName(in_summoner_name, in_region)
        if summoner:
            img_prof_icon = self._resource_man.getProfileIconImage(summoner['profileIconId'])
            display.display(img_prof_icon.resize((50,50)))
            print(f"Summoner: {summoner['name']} (level:{summoner['summonerLevel']})")
            summoner_league = self._riot_api_caller.runGetSummonerLeagueBySummonerId(summoner['id'], in_region)
            if summoner_league:
                for lidx in range(len(summoner_league)):
                    img_tier = self._resource_man.getRankEmblem(summoner_league[lidx]['tier'])
                    display.display(img_tier.resize((50,50)))
                    print(f"Queue: {summoner_league[lidx]['queueType']} Rank: {summoner_league[lidx]['tier']}-{summoner_league[lidx]['rank']}")
    
    def collectMatchData(self):
        entries = runGetLeagueEntries('RANKED_FLEX_SR','DIAMOND','I','euw1')
        return entries
        
ala = ALAMain()
ala.getSummonerInfo('deamonpog','na1')
entries = ala.collectMatchData()
print(entries)

In [ ]:
root_dir = "D:\\PROJECTS\\RIOT\\"
riot_api_key_file = "riot_api_key.json"
data_dragon_root = "DataDragon/dragontail-10.16.1/"

In [ ]:
API_KEY = ""
with open(path.join(root_dir, riot_api_key_file)) as f:
    API_KEY = json.loads(f.readlines()[0])["API_KEY"]
    
print(f"Key reading was {'not ' if (API_KEY == '') else ''}successful.")

In [ ]:
champ_data = []
id_to_champ = {}

with open(path.join(root_dir, data_dragon_root, f"10.16.1/data/en_US/champion.json"), encoding='utf-8') as f:
    r = f.readlines()[0]
    champ_data = json.loads(r)['data']

for ch in champ_data:
    id_to_champ[int(champ_data[ch]['key'])] = ch

In [ ]:
def runGetSubdomain(in_subdomain_with_params, in_region, in_additional_params_str = ''):
    get_url = f"https://{in_region}.api.riotgames.com/lol/{in_subdomain_with_params}?api_key={API_KEY}{in_additional_params_str}"
    return requests.get(get_url)
    
def runGetSummonerByName(in_summoner_name, in_region):
    subdomain_with_params = f"summoner/v4/summoners/by-name/{in_summoner_name}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetSummonerLeagueBySummonerId(in_encrypted_summoner_id, in_region):
    subdomain_with_params = f"league/v4/entries/by-summoner/{in_encrypted_summoner_id}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetMatchListByAccount(in_encrypted_account_id, in_region):
    subdomain_with_params = f"match/v4/matchlists/by-account/{in_encrypted_account_id}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetMatchInfoByMatchId(in_match_id, in_region):
    subdomain_with_params = f"match/v4/matches/{in_match_id}"
    return runGetSubdomain(subdomain_with_params, in_region)
    
def runGetMatchTimelineDataByGameID(in_match_id, in_region):
    subdomain_with_params = f"match/v4/timelines/by-match/{in_match_id}"
    return runGetSubdomain(subdomain_with_params, in_region)

def runGetChampionMasteriesBySummonerId(in_encrypted_summoner_id, in_region):
    subdomain_with_params = f"champion-mastery/v4/champion-masteries/by-summoner/{in_encrypted_summoner_id}"
    return runGetSubdomain(subdomain_with_params, in_region)


In [ ]:
in_sumName = 'Anduru'
in_region = 'euw1'

get_response = runGetSummonerByName(in_sumName,in_region)
if get_response.status_code == 200:
    summoner_info = get_response.json()
    img_prof_icon = Image.open(path.join(root_dir, data_dragon_root, f"10.16.1/img/profileicon/{summoner_info['profileIconId']}.png"))
    display.display(img_prof_icon.resize((50,50)))
    print(f"Summoner: {summoner_info['name']} (level:{summoner_info['summonerLevel']})")
else:
    print(f"Response code: {get_response.status_code}")

get_response = runGetSummonerLeagueBySummonerId(summoner_info['id'],in_region)
if get_response.status_code == 200:
    summoner_league = get_response.json()
    for lidx in range(len(summoner_league)):
        img_tier = Image.open(path.join(root_dir, f"Assets/ranked-emblems/Emblem_{(summoner_league[lidx]['tier']).capitalize()}.png"))
        display.display(img_tier.resize((50,50)))
        print(f"Queue: {summoner_league[lidx]['queueType']} Rank: {summoner_league[lidx]['tier']}-{summoner_league[lidx]['rank']}")
else:
    print(f"Response code: {get_response.status_code}")
    
get_response = runGetChampionMasteriesBySummonerId(summoner_info['id'],in_region)
if get_response.status_code == 200:
    summoner_mastery = get_response.json()
    for chidx in range(len(summoner_mastery)):
        ch = id_to_champ[summoner_mastery[chidx]['championId']]
        img_champion = Image.open(path.join(root_dir, data_dragon_root, f"10.16.1/img/sprite/{champ_data[ch]['image']['sprite']}"))
        display.display(img_champion.crop((champ_data[ch]['image']['x'],champ_data[ch]['image']['y'],champ_data[ch]['image']['x'] + champ_data[ch]['image']['w'],champ_data[ch]['image']['y'] + champ_data[ch]['image']['h'])).resize((50,50)))
        print(f"Mastery: {summoner_mastery[chidx]['championLevel']} with Mastery Points: {summoner_mastery[chidx]['championPoints']}")
        if chidx >= 10:
            break
else:
    print(f"Response code: {get_response.status_code}")
    
get_response = runGetMatchListByAccount(summoner_info['accountId'],'euw1')
if get_response.status_code == 200:
    match_list = get_response.json()
    print(f"Number of matches: {len(match_list['matches'])}")
else:
    print(f"Response code: {get_response.status_code}")
          
get_response = runGetMatchInfoByMatchId(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_info = get_response.json()
    print(f"The Game mode: {match_info['gameMode']}")
else:
    print(f"Response code: {get_response.status_code}")

get_response = runGetMatchTimelineDataByGameID(match_list['matches'][0]['gameId'],'euw1')
if get_response.status_code == 200:
    match_data = get_response.json()
    print(f"Number of frames: {len(match_data['frames'])}")
else:
    print(f"Response code: {get_response.status_code}")

In [ ]:
match_data['frames'][4]['events']

In [ ]:
summoner_info

In [ ]:
match_info['participantIdentities'][7]

In [ ]:
summoner_league

In [ ]:
id_to_champ['67']